# layer.py对照
检查quantize和layers能否保证在各个模式下完全一致，包括structure_forward。、

首先，利用暂存的Lenet权重，提取第一层并保存，并记录相关信息，同时随机生成一组input作为简单的测试
LeNet的测试已经通过，在三种不同的模式下都能保证一致

接着测试VGG8的第二个卷积层，这样可以保证存在被划分的结构

In [ ]:
import torch
weights = torch.load("../zoo/cifar10_vgg8_params.pth")
# print(weights.keys())
layer = dict()
for name, params in weights.items():
    if name.startswith("layer_list.3"):
        layer[name[len("layer_list.3."):]] = params
# save layer1
torch.save(layer, "../zoo/cifar10_test_layer.pth")
# print(layer["bit_scale_list"])
# save input
input = torch.clamp(torch.randn(1, 128, 4, 4), 0, 1) * \
    (255 * layer["bit_scale_list"][0, 1].item())
torch.save(input, "../zoo/cifar10_test_input.pth")

In [ ]:
import torch
import MNSIM.Interface.quantize as quantize
from MNSIM.Interface.quantize import QuantizeLayer
hardware_config = {
    "xbar_size": 512,
    "input_bit": 2,
    "weight_bit": 1,
    "quantize_bit": 10,
}
layer_config = {
    "type": "conv",
    "in_channels": 128,
    "out_channels": 128,
    "kernel_size": 3,
}
quantize_config = {
    "weight_bit": 9,
    "activation_bit": 9,
    "point_shift": -2,
}
layer = QuantizeLayer(hardware_config, layer_config, quantize_config)
layer.load_state_dict(torch.load("../zoo/cifar10_test_layer.pth"))
input = torch.load("../zoo/cifar10_test_input.pth")
# forward
quantize.last_activation_scale = 4.8407 * 1e-3
quantize.last_activation_bit = 9
layer.eval()
output = layer.forward(input, method="SINGLE_FIX_TEST")
print(output)
torch.save(output, "quantize.pth")

In [ ]:
import torch
from MNSIM.Interface.layer import BaseWeightLayer
layer_ini = {
    "layer": {
        "in_channels": 128,
        "out_channels": 128,
        "kernel_size": 3,
    },
    "quantize": {
        "input": 9,
        "weight": 9,
        "output": 9,
    },
    "hardware": {
        "xbar_row": 512,
        "cell_bit": 1,
        "dac_bit": 2,
        "adc_bit": 10,
        "point_shift": -2,
    }
}
layer = BaseWeightLayer.get_class_("conv")(layer_ini)
layer.load_state_dict(torch.load("../zoo/cifar10_test_layer.pth"), strict=False)
input = torch.load("../zoo/cifar10_test_input.pth")
# forward
layer.eval()
input_config = [torch.FloatTensor([9, 4.8407 * 1e-3])]
output = layer.forward(input, method="SINGLE_FIX_TEST", input_config=input_config)
print(output)
torch.save(output, "layer.pth")


In [ ]:
import torch
a = torch.load("layer.pth")
b = torch.load("quantize.pth")
print(torch.max(a-b))

## 检查model是否正确，并和之前的结果做对照

In [ ]:
import torch
input = torch.clamp(torch.randn(1, 3, 32, 32), 0, 1)
torch.save(input, "model_input.pth")

In [ ]:
import torch
from MNSIM.Interface.dataset import ClassificationBaseDataset
from MNSIM.Interface.model import BaseModel

dataset_ini = {
    "TRAIN_BATCH_SIZE": 128,
    "TRAIN_NUM_WORKERS": 4,
    "TEST_BATCH_SIZE": 100,
    "TEST_NUM_WORKERS": 4,
}
model_config_path = "lenet.yaml"
dataset = ClassificationBaseDataset.get_class_("cifar10")(dataset_ini)
model = BaseModel.get_class_("yaml")(model_config_path, dataset.get_dataset_info())
model.load_change_weights(torch.load("../zoo/cifar10_lenet_params.pth"))
# input
input = torch.load("model_input.pth")
model.eval()
output = model.forward(input, method="SINGLE_FIX_TEST")
for name, param in model.named_parameters():
    print(name)
print(output)
torch.save(output, "model_output.pth")

In [ ]:
import torch
from MNSIM.Interface.network import get_net
net = get_net(cate="lenet", num_classes=10)
net.load_state_dict(torch.load("../zoo/cifar10_lenet_params.pth"))
# input
input = torch.load("model_input.pth")
net.eval()
output = net.forward(input, method="SINGLE_FIX_TEST")
print(output)
torch.save(output, "net_output.pth")

In [ ]:
import torch
a = torch.load("model_output.pth")
b = torch.load("net_output.pth")
print(a)
print(b)
print(torch.max(a-b))

In [ ]:
import torch
for i in range(17):
    a = torch.load(f"modle_list_{i}.pth")
    b = torch.load(f"net_list_{i}.pth")
    print(torch.max(torch.abs(a-b)))